# Settings

In [5]:
import mne, os, glob, pickle
import numpy as np
import pandas as pd
import matplotlib
import itertools
matplotlib.use('Qt5Agg')

expt = 'SemRep'
main_dir = f'/Volumes/Server/NEUROLING/Projects/{expt}/data' # dataset directory
meg_dir = os.path.join(main_dir, 'meg')
mri_dir = os.path.join(main_dir, 'mri')
os.chdir(main_dir)
parts = [1,2]

mne.set_log_level(verbose=30)

import matplotlib.pyplot as plt

In [6]:
subjects =['R1441','R1701','R1834','R1954','R1974','R1985','R1993','R1994','R1998','R2021','R2029','R2031','R2041','R1439','R2148', 'R1830', 'R2149']
parts = [1,2]

## Subject specific settings

In [8]:
subj = subjects[0]
part = parts[0]

In [9]:
# Create all nec directories
cache_dir = main_dir + f'/meg/{subj}/cache/'

# figure dir
if not os.path.exists(f'figures/{subj}/'):
    os.makedirs(f'figures/{subj}/')
# source dir
if not os.path.exists(f'meg/{subj}/source/'):
            os.makedirs(f'meg/{subj}/source/')
# whitened evoked dir
if not os.path.exists(f'figures/{subj}/whitened/'):
    os.makedirs(f'figures/{subj}/whitened/')

In [10]:
# and all nec filenames

trans_fname = mne.read_trans(f'meg/{subj}/{subj}-trans-{part}.fif') # reading in coregistration transformation
bem_fname = os.path.join(mri_dir, f'{subj}/bem/{subj}-inner_skull-bem-sol.fif')
src_fname = os.path.join(mri_dir, f'{subj}/bem/{subj}-ico-4-src.fif')
cov_fname = f'meg/{subj}/source/{subj}_part{part}-cov.fif'
fwd_fname = f'meg/{subj}/source/{subj}_part{part}-fwd.fif'
inv_fname = f'meg/{subj}/source/{subj}_part{part}-inv.fif'  

# Create evoked

In [11]:
# adapted from a larger subject loop

print(f"STCs for subj = {subj}")

cleandata = mne.read_epochs(os.path.join(meg_dir, subj, 'preproc', f'{expt}_{subj}_final_part{part}-epo.fif'))
        
# crop the epochs (only nec for my experiment)
cleandata = cleandata.crop(0,0.65)
        
# select and define all of the conditions       
info = cleandata.info
cleandata.metadata = cleandata.metadata.replace(['U', 'H', 'P'], ['unam', 'hom', 'pol'])
print('Data read in...')

POS = list(cleandata.metadata['POS'].unique())
ST = list(cleandata.event_id.keys())
context = list(cleandata.metadata['context'].unique())
detailed_cond = list(itertools.product(ST, context,POS)) 

print(f'Conditions generated... Creating evoked responses for {subj}')

evoked = []
for c1,c2,c3 in detailed_cond:
        query_string = f"`semantic type` == '{c1}' and context == '{c2}' and POS == '{c3}'"
        cond = cleandata[query_string].average() # average over conditions
        cond.comment = c1 +'-'+c2+'-'+c3
        if not os.path.exists(f'figures/{subj}/{subj}_evoked_part{part}-{cond.comment}.png'): # plot the evoked response
            cond_plot = cond.plot_joint(title= cond.comment, show=False);
            cond_plot.savefig(f'figures/{subj}/{subj}_evoked_part{part}-{cond.comment}.png')
            plt.close()
        evoked.append(cond)

        
print('Full condition list finished...')

STCs for subj = R1441
Data read in...
Conditions generated... Creating evoked responses for R1441
Full condition list finished...


### Sanity check: plot all evoked for one subj

In [13]:
all_evokeds = mne.combine_evoked(evoked, weights='equal')
evoked_plot = all_evokeds.plot_joint(show=False)
evoked_plot.comment = 'grand avg'
evoked_plot.savefig(f'figures/{subj}/{subj}_allevoked_part{part}.png')
plt.close()

# Create Source Timecourses

## Covariance and whitened data

In [15]:
# compute the noise covariance and plot whitened data as a sanity check
baseline = (-0.1, 0)
cov = mne.compute_covariance(cleandata, tmin=baseline[0], tmax=baseline[1], method=['shrunk', 'diagonal_fixed', 'empirical'])
#cov.save(cov_fname, overwrite=True)

for ev in evoked:
    fig = ev.plot_white(cov, show=False)
    fig.savefig(f'figures/{subj}/whitened/whitened-{cond.comment}_part{part}.png')
print('Full condition list finished...')

/var/folders/f_/8vg268vd0qqb5s9vcd342s7r0000gr/T/ipykernel_39048/374009836.py:3: RuntimeWarning: Too few samples (required : 790 got : 445), covariance estimate may be unreliable
  cov = mne.compute_covariance(cleandata, tmin=baseline[0], tmax=baseline[1], method=['shrunk', 'diagonal_fixed', 'empirical'])


Full condition list finished...
Full condition list finished...
Full condition list finished...
Full condition list finished...
Full condition list finished...
Full condition list finished...
Full condition list finished...
Full condition list finished...
Full condition list finished...
Full condition list finished...
Full condition list finished...
Full condition list finished...
Full condition list finished...
Full condition list finished...
Full condition list finished...
Full condition list finished...
Full condition list finished...
Full condition list finished...


## Create source space

In [16]:
if not os.path.isfile(src_fname):
    print('Setting up source space')
    src = mne.setup_source_space(subject=subj, spacing='ico4', subjects_dir=mri_dir)
    src.save(src_fname, overwrite=True)
else:
    print('src for subj = %s already exists, loading file...' %subj)
    src = mne.read_source_spaces(src_fname)
    print('Source space completed!')

src for subj = R1441 already exists, loading file...
Source space completed!


## BEM model

In [ ]:
if not os.path.isfile(bem_fname):
    conductivity = (0.3,) # for single layer
    model = mne.make_bem_model(subject=subj, ico=4, conductivity=conductivity, subjects_dir=mri_dir)
    bem = mne.make_bem_solution(model)
    mne.write_bem_solution(bem_fname, bem)
    print ('Done. File saved.')

## Forward solution

In [17]:
print('Getting forward solution')
if not os.path.isfile(fwd_fname):
    print ('forward solution for subj = %s does not exist, creating file.' %subj)
    fwd = mne.make_forward_solution(info=info, trans=trans_fname, src=src, bem=bem_fname, ignore_ref=True)
    mne.write_forward_solution(fwd_fname, fwd, overwrite=True)
else:
    print('fwd for subj = %s already exists, loading file...' %subj)
    fwd = mne.read_forward_solution(fwd_fname)
    print('Done.')

Getting forward solution
fwd for subj = R1441 already exists, loading file...
Done.


## Inverse operator

In [19]:
print ('Getting inverse operator')
cov = mne.read_cov(cov_fname)
fixed = False # False: Ignoring dipole direction. Makes them perpendicular to surface. Creates unsigned data when pick_org is None.
SNR = 3 # 3 for evoked, 2 for single trial analysis
if fixed: # for signed data
    fwd = mne.convert_forward_solution(fwd, surf_ori=True)

inv = mne.minimum_norm.make_inverse_operator(info, fwd, cov, depth=0.8, loose=0.2, fixed=fixed) # loose sets how offset from perpendicular the dipoles are
lambda2 = 1.0 / SNR ** 2.0 # regularization parameter
print('Done!')

Getting inverse operator
Done!


## Create stcs

In [20]:
print(f'Creating morphed stcs for subject {subj}')
for ev in evoked:
    stc = mne.minimum_norm.apply_inverse(ev, inv, lambda2=lambda2, method='dSPM')
    stc.subject = subj
    # morphing stcs to the fsaverage using precomputed matrix method:
    morph = mne.compute_source_morph(stc, subject_from=subj, subject_to='fsaverage', subjects_dir=mri_dir, spacing = 4)
    stc_dir = f'full_stc/{ev.comment}'
    stc_fname = f'{subj}_part{part}_{ev.comment}_dSPM'
    print(f'Morphing {stc_fname} to fsaverage...')
    stc_fsaverage = morph.apply(stc)
    if not os.path.exists(stc_dir):
        os.makedirs(stc_dir)
    #stc_fsaverage.save(os.path.join(stc_dir, stc_fname), overwrite=True)

print(f'Finished subject {subj} \n')

Creating morphed stcs for subject R1441
Morphing R1441_part1_unam-Null-noun_dSPM to fsaverage...
Morphing R1441_part1_unam-Null-verb_dSPM to fsaverage...
Morphing R1441_part1_unam-Emb-noun_dSPM to fsaverage...
Morphing R1441_part1_unam-Emb-verb_dSPM to fsaverage...
Morphing R1441_part1_unam-Min-noun_dSPM to fsaverage...
Morphing R1441_part1_unam-Min-verb_dSPM to fsaverage...
Morphing R1441_part1_pol-Null-noun_dSPM to fsaverage...
Morphing R1441_part1_pol-Null-verb_dSPM to fsaverage...
Morphing R1441_part1_pol-Emb-noun_dSPM to fsaverage...
Morphing R1441_part1_pol-Emb-verb_dSPM to fsaverage...
Morphing R1441_part1_pol-Min-noun_dSPM to fsaverage...
Morphing R1441_part1_pol-Min-verb_dSPM to fsaverage...
Morphing R1441_part1_hom-Null-noun_dSPM to fsaverage...
Morphing R1441_part1_hom-Null-verb_dSPM to fsaverage...
Morphing R1441_part1_hom-Emb-noun_dSPM to fsaverage...
Morphing R1441_part1_hom-Emb-verb_dSPM to fsaverage...
Morphing R1441_part1_hom-Min-noun_dSPM to fsaverage...
Morphing R144